In [1]:
# Import Dependencies
import requests
import json
import pandas as pd
import time
import numpy as np
from config import yelp_key

In [2]:
# API Base URL
base_url = "https://api.yelp.com/v3/businesses/search?"

In [3]:
# API Headers
headers = {
    "accept" : "application/json",
    "Authorization" : "Bearer " + yelp_key 
}

In [4]:
# Create List of Categories (manually located)
categories = ['beaches','hanggliding','horsebackriding',
             'hiking','hot_air_balloons',
              'paddleboarding','parasailing',
              'sailing','snorkeling','ziplining', 
              'spas','hotsprings','massage','tours'
             ]

len(categories)

14

In [6]:
# Import Lat/Long Database
csv_path = "../Resources/CitiesWGeolocation.csv"
locations_df = pd.read_csv(csv_path)
locations_df

,Unnamed: 0,City,State/Province,Country,Latitude,Longitude
0,0,Lexington,Kentucky,United States,38.046407,-84.497039
1,1,San Diego,California,United States,32.717420,-117.162773
2,2,Cook Islands,NaN,Cook Islands,-19.996972,-157.785871
3,3,Park City,Utah,United States,40.646092,-111.497996
4,4,Newcastle Upon Tyne,England,United Kingdom,54.973847,-1.613157
...,...,...,...,...,...,...
223,223,Beirut,NaN,Lebanon,33.895920,35.478430
224,224,Zurich,NaN,Switzerland,47.374449,8.541042
225,225,Geneva,NaN,Switzerland,46.201756,6.146601
226,226,Valletta,NaN,Malta,35.898982,14.513676


In [7]:
# Normalize capitalization
locations_df = locations_df.rename(columns = {"Latitude": "latitude","Longitude":"longitude","Unnamed: 0":"locationID"})

In [8]:
# Gather Lists of Latitudes and Longitudes
latitudes_list = []
longitudes_list = []
location_list = []
for i in range(locations_df.shape[0]):
    latitudes_list.append(locations_df.loc[i].at['latitude'])
    longitudes_list.append(locations_df.loc[i].at['longitude'])
    location_list.append(locations_df.loc[i].at['locationID'])
print(latitudes_list)
print(longitudes_list)

[38.0464066, 32.7174202, -19.99697155, 40.646092100000004, 54.97384739999999, -27.468968199999996, -28.0402165, 34.244058700000004, -36.718804999999996, 49.5041747, 42.64873625, 37.2395367, -32.728465, 17.223472100000002, 25.265347100000003, 50.67108245, 21.721746, 36.508976000000004, 32.079007399999995, -16.484598300000002, -33.928992, -19.1421421, -33.934444, -28.648333299999997, -45.0321923, 33.77217945, -6.166490799999999, -20.2759451, 45.437190799999996, 19.70318225, -4.6574976999999995, -28.002373100000003, 44.4643768, 31.6258257, 37.9374939, 49.8879177, 43.10656029999999, 30.2711224, 15.2214956, 43.4832523, -33.953177600000004, 26.9154576, 46.603353999999996, 13.1500331, 20.169626800000003, 51.53882410000001, 60.39430550000001, 24.578720999999998, -34.61341495, 34.862942600000004, 18.185050699999998, 12.51756625, -35.14181285, -34.4175, 46.7985624, -33.8611665, -16.9206657, 44.958452799999996, -26.6544338, -34.427808299999995, 51.08668970000001, 44.279621, -37.8142454, -31.95589

In [9]:
# Gather List of URLs
url_list = []
lat_lon_list = []
cat_list = []
location_ID_list = []
for i in range(len(latitudes_list)):
        for x in range(len(categories)):
            latitude = latitudes_list[i]
            longitude = longitudes_list[i]
            category = categories[x]
            location = location_list[i]
            # Limit radius to appx 15 miles from the coordinates
            complete_url = base_url + "latitude=" + str(latitude) + "&longitude=" + str(longitude) + "&radius=40000&categories=" + category + "&sort_by=rating"
            url_list.append(complete_url)
            lat_lon_list.append([latitude,longitude])
            cat_list.append(category)
            location_ID_list.append(location)
# Confirm list of API calls is less than 5,000 daily limit
print(len(url_list))
url_list

3192


['https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=beaches&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=hanggliding&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=horsebackriding&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=hiking&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=hot_air_balloons&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=paddleboarding&sort_by=rating',
 'https://api.yelp.com/v3/businesses/search?latitude=38.0464066&longitude=-84.4970393&radius=40000&categories=parasailing&sort_by=rating',
 'https://api.yelp.com/v

In [16]:
# Create API Calls (separate the blank list to a new cell so we can do numerous calls if needed)
data_list=[]

In [17]:
record_count = 1
set_count = 1
url_errors = []


for i in range(len(url_list)):
    if (i%14 == 0 and i >=14):
#         set_count += 1
        record_count = 1
        
    print("processing Record %s | Location: %s / %s" %(record_count,location_ID_list[i],location_ID_list.nunique()))    
    
    record_count += 1

    response = requests.get(url_list[i],headers=headers)
    data = response.json()
    try:
        # Calculate Average Rating and Review Count
        rating_sum = 0
        review_sum = 0
        # setting the range with an initial number so it can be changed if an error pops up midway through
        for x in range(len(data["businesses"])):
            rating_sum += data["businesses"][x]["rating"]
            review_sum += data["businesses"][x]["review_count"]
        rating_avg = rating_sum / len(data["businesses"])
        review_avg = review_sum / len(data["businesses"])
    
        # Add the desired data to a dictionary
        data_dict = {
            "locationID" : location_ID_list[i],
            "latitude" : lat_lon_list[i][0] ,
            "longitude": lat_lon_list[i][1],
            "category" : cat_list[i],
            "results_total" : len(data["businesses"]),
            "avg_rating" : rating_avg,
            "avg_review_count" : review_avg
        }
        # Append the dictionary to a list
        data_list.append(data_dict)
        time.sleep(1)
    except:
        print("********URL error: Skipping...********")
        print(data)
        try:
            url_errors.append({"locationID":location_ID_list[i],"url":url_list[i],"latitude":data["region"]["center"]["latitude"],
                           "longitude":data["region"]["center"]["longitude"]})
        except:
            url_errors.append({"locationID":location_ID_list[i],"url":url_list[i],"latitude":"NAN",
                           "longitude":"NAN"})
        pass

processing Record 1 | Location: 0 / 3192
processing Record 2 | Location: 0 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -84.4970393, 'latitude': 38.0464066}}}
processing Record 3 | Location: 0 / 3192
processing Record 4 | Location: 0 / 3192
processing Record 5 | Location: 0 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -84.4970393, 'latitude': 38.0464066}}}
processing Record 6 | Location: 0 / 3192
processing Record 7 | Location: 0 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -84.4970393, 'latitude': 38.0464066}}}
processing Record 8 | Location: 0 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -84.4970393, 'latitude': 38.0464066}}}
processing Record 9 | Location: 0 / 3192
********URL error: Skipping...********
{'businesses': [

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 153.02349909999998, 'latitude': -27.468968199999996}}}
processing Record 8 | Location: 5 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 153.02349909999998, 'latitude': -27.468968199999996}}}
processing Record 9 | Location: 5 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 153.02349909999998, 'latitude': -27.468968199999996}}}
processing Record 10 | Location: 5 / 3192
processing Record 11 | Location: 5 / 3192
processing Record 12 | Location: 5 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 153.02349909999998, 'latitude': -27.468968199999996}}}
processing Record 13 | Location: 5 / 3192
processing Record 14 | Location: 5 / 3192
processing Record 1 | Location: 6 / 3192
processing Record 2 | Locat

processing Record 14 | Location: 9 / 3192
processing Record 1 | Location: 10 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 18.094680152605108, 'latitude': 42.64873625}}}
processing Record 2 | Location: 10 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 18.094680152605108, 'latitude': 42.64873625}}}
processing Record 3 | Location: 10 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 18.094680152605108, 'latitude': 42.64873625}}}
processing Record 4 | Location: 10 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 18.094680152605108, 'latitude': 42.64873625}}}
processing Record 5 | Location: 10 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 18.094680152605108, 'latitude': 42.

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -61.9554608, 'latitude': 17.223472100000002}}}
processing Record 13 | Location: 13 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -61.9554608, 'latitude': 17.223472100000002}}}
processing Record 14 | Location: 13 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -61.9554608, 'latitude': 17.223472100000002}}}
processing Record 1 | Location: 14 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 55.292491399999996, 'latitude': 25.265347100000003}}}
processing Record 2 | Location: 14 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 55.292491399999996, 'latitude': 25.265347100000003}}}
processing Record 3 | Location: 14 / 3192
********URL er

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -4.885619999999999, 'latitude': 36.508976000000004}}}
processing Record 8 | Location: 17 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -4.885619999999999, 'latitude': 36.508976000000004}}}
processing Record 9 | Location: 17 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -4.885619999999999, 'latitude': 36.508976000000004}}}
processing Record 10 | Location: 17 / 3192
processing Record 11 | Location: 17 / 3192
processing Record 12 | Location: 17 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -4.885619999999999, 'latitude': 36.508976000000004}}}
processing Record 13 | Location: 17 / 3192
processing Record 14 | Location: 17 / 3192
processing Record 1 | Location: 18 / 3192
processing Record 2 | L

...

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -68.1336229, 'latitude': -16.4955455}}}
processing Record 4 | Location: 211 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -68.1336229, 'latitude': -16.4955455}}}
processing Record 5 | Location: 211 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -68.1336229, 'latitude': -16.4955455}}}
processing Record 6 | Location: 211 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -68.1336229, 'latitude': -16.4955455}}}
processing Record 7 | Location: 211 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -68.1336229, 'latitude': -16.4955455}}}
processing Record 8 | Location: 211 / 3192
********URL error: Skipping...********
{'businesses': [], 't

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 34.7818064, 'latitude': 32.0852997}}}
processing Record 13 | Location: 214 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 34.7818064, 'latitude': 32.0852997}}}
processing Record 14 | Location: 214 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 34.7818064, 'latitude': 32.0852997}}}
processing Record 1 | Location: 215 / 3192
processing Record 2 | Location: 215 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -3.1883749, 'latitude': 55.9533456}}}
processing Record 3 | Location: 215 / 3192
processing Record 4 | Location: 215 / 3192
processing Record 5 | Location: 215 / 3192
processing Record 6 | Location: 215 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'

processing Record 9 | Location: 218 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 10.7389701, 'latitude': 59.913330099999996}}}
processing Record 10 | Location: 218 / 3192
processing Record 11 | Location: 218 / 3192
processing Record 12 | Location: 218 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 10.7389701, 'latitude': 59.913330099999996}}}
processing Record 13 | Location: 218 / 3192
processing Record 14 | Location: 218 / 3192
processing Record 1 | Location: 219 / 3192
processing Record 2 | Location: 219 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -4.250168700000001, 'latitude': 55.861155000000004}}}
processing Record 3 | Location: 219 / 3192
processing Record 4 | Location: 219 / 3192
processing Record 5 | Location: 219 / 3192
********URL error: Skipping...********
{'businesses': [], 'to

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 35.478429999999996, 'latitude': 33.8959203}}}
processing Record 5 | Location: 223 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 35.478429999999996, 'latitude': 33.8959203}}}
processing Record 6 | Location: 223 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 35.478429999999996, 'latitude': 33.8959203}}}
processing Record 7 | Location: 223 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 35.478429999999996, 'latitude': 33.8959203}}}
processing Record 8 | Location: 223 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': 35.478429999999996, 'latitude': 33.8959203}}}
processing Record 9 | Location: 223 / 3192
********URL error: Skipping...

********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -21.9422367, 'latitude': 64.145981}}}
processing Record 5 | Location: 227 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -21.9422367, 'latitude': 64.145981}}}
processing Record 6 | Location: 227 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -21.9422367, 'latitude': 64.145981}}}
processing Record 7 | Location: 227 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -21.9422367, 'latitude': 64.145981}}}
processing Record 8 | Location: 227 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -21.9422367, 'latitude': 64.145981}}}
processing Record 9 | Location: 227 / 3192
********URL error: Skipping...********
{'businesses': [], 'total': 0, 

In [18]:
# convert the list of dictionaries to a dataframe
yelp_df = pd.DataFrame(data_list)   
yelp_df.head()

,locationID,latitude,longitude,category,results_total,avg_rating,avg_review_count
0,0,38.046407,-84.497039,beaches,2,4.75,3.500
1,0,38.046407,-84.497039,horsebackriding,16,4.50,13.375
2,0,38.046407,-84.497039,hiking,8,4.50,8.500
3,0,38.046407,-84.497039,paddleboarding,2,4.75,16.500
4,0,38.046407,-84.497039,ziplining,20,5.00,7.900


In [19]:
# Convert the list of errors into a dataframe to view any similarities
errors_df = pd.DataFrame(url_errors)
errors_df.head()

,locationID,url,latitude,longitude
0,0,https://api.yelp.com/v3/businesses/search?lati...,38.046407,-84.497039
1,0,https://api.yelp.com/v3/businesses/search?lati...,38.046407,-84.497039
2,0,https://api.yelp.com/v3/businesses/search?lati...,38.046407,-84.497039
3,0,https://api.yelp.com/v3/businesses/search?lati...,38.046407,-84.497039
4,0,https://api.yelp.com/v3/businesses/search?lati...,38.046407,-84.497039


In [20]:
# determine if there are any missing reviews. All 14 categories are accounted for
yelp_df["category"].value_counts()

ziplining           168
tours               145
massage             141
spas                140
beaches             125
hiking              124
horsebackriding     115
sailing              77
paddleboarding       57
hot_air_balloons     47
snorkeling           29
hanggliding          19
hotsprings           13
parasailing          11
Name: category, dtype: int64

In [21]:
# Check how many categories successfully returned for each of the locations that returned.
# We are missing many locations, as the length should be 237. None of our locations have the full 14 activities either.
yelp_df["latitude"].nunique()

167

In [22]:
# Separate out all of the values that have less than 5 actual activities returned successfully
activity_df = pd.DataFrame(yelp_df.loc[(yelp_df["category"] != "transportation") & (yelp_df["category"] !="restaurants") &
                                       (yelp_df["category"] != "hotels")         & (yelp_df["category"] != "nightlife")])
value_counts = pd.DataFrame(activity_df[["locationID","latitude","longitude"]].groupby(["latitude","longitude"]).size().sort_values(ascending=False))
value_counts = value_counts.rename(columns={0:"activityCount","longitude":"test"})
value_counts = value_counts.loc[(value_counts["activityCount"] < 5)]
value_counts = value_counts.reset_index(inplace=False)
value_counts.head()

,latitude,longitude,activityCount
0,49.504175,-115.062867,4
1,-19.142142,146.833549,4
2,-33.953178,115.076937,4
3,12.517566,-69.981864,3
4,-35.141813,150.391646,3


In [ ]:
# Filter the list of results to only include locations with five or more actual activities
longitudes_list = value_counts["longitude"].tolist()
check_five = yelp_df[~yelp_df["longitude"].isin(longitudes_list)]
check_five

In [ ]:
# Create a dataframe of all null values in the yelp dataframe
null_df = pd.merge(check_five,locations_df,how="right",on=["latitude","longitude","locationID"])
null_df = null_df[null_df["category"].isna()]
null_df

In [ ]:
# Export list of all locations with 5+ activities
yelp_df.to_csv("../Resources/yelpdata.csv")

In [ ]:
# Export list of all locations with 0-4 activities
null_df.to_csv("../Resources/missingActivities.csv")

In [ ]:
check_five['category'].unique()